# OLS

The linear method makes use of the log backward return (log price difference) to predict foward return.

Training:
1) Ridge regression: on 30 features
2) PC regression: pca on 30 features then perform ols

Feature: 10 stocks, each with 3 backward return (say, 3min, 7min, 10min, see correlation to decide)

Response: 10 stocks' 30min forward return. 

Groups: [1,4,5,6,8],[0,2,3,7,9]

## Data Preparation

In [1]:
import os
import datetime
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 

log_pr = pd.read_pickle("../data/log_price.df")
volu = pd.read_pickle("../data/volume_usd.df")

## Feature Engineering

In [2]:
def rsi(close_delta, periods=20, ema=True):
    """
    Returns a pd.Series with the relative strength index.
    """
    close_delta = close_delta.diff()

    # Make two series: one for lower closes and one for higher closes
    up = close_delta.clip(lower=0)
    down = -1 * close_delta.clip(upper=0)
    
    if ema == True:
	    # Use exponential moving average
        ma_up = up.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
        ma_down = down.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
    else:
        # Use simple moving average
        ma_up = up.rolling(window = periods, adjust=False).mean()
        ma_down = down.rolling(window = periods, adjust=False).mean()
        
    rsi = ma_up / ma_down
    rsi = 100 - (100/(1 + rsi))
    return rsi


def wide_format(df):
    df_= df.reset_index(level=['stock']).sort_index()
    df_ = df_.pivot(columns ='stock')
    df_.columns = df_.columns.get_level_values(0) + '_' +  [str(x) for x in df_.columns.get_level_values(1)]

    return df_


def get_feature_train(log_pr, volu, x_begin_idx, x_end_idx, y_begin_idx, 
                        grp_idx=None, rm_outlier=False, print_cor=True):
    """
    Input:
    log_pr (pdSeries): train set
    volu (pdSeries): train set
    x_begin_idx (pdIndex): to truncate the NaNs
    grp_idx (dict): key is group idx, value is list of stock idx

    Returns:
    feature_dict (dict): key is group idx, value is a tuple of feature matrix and response
    """

    log_pr_df = log_pr.reset_index().melt(id_vars=['timestamp'])
    log_pr_df.columns = ['timestamp', 'stock', 'log_pr']
    log_pr_df = log_pr_df.set_index(['timestamp', 'stock']).sort_index()

    volu_df = volu.reset_index().melt(id_vars=['timestamp'])
    volu_df.columns = ['timestamp', 'stock', 'volu']
    volu_df = volu_df.set_index(['timestamp', 'stock']).sort_index()

    features = pd.DataFrame(index=log_pr_df.index)
    # features['trend'] = np.ones(log_pr_df.shape[0])

    # log_pr feature
    for i in [30]:
        features['log_pr_{}'.format(i)] = -log_pr_df.groupby(level='stock').log_pr.diff(i)

    k_period = 40
    d_period = 3
    ma_max = lambda x: x.rolling(k_period).max()
    ma_min = lambda x: x.rolling(k_period).min()
    mad = lambda x: x.rolling(d_period).mean()
    # msd = lambda x: x.rolling(d_period).sum()

    features['pr_min_40'] = log_pr_df.groupby(level='stock').log_pr.apply(ma_min)
    features['pr_max_40'] = log_pr_df.groupby(level='stock').log_pr.apply(ma_max)

    features['pr_so_40'] = (log_pr_df.log_pr - features['pr_min_40'])*100 / (features['pr_max_40'] - features['pr_min_40'])
    features['pr_so_40d3'] = features.groupby(level='stock').pr_so_40.apply(mad)

    # STD of log price
    for i in [10,30]:
        std = lambda x: x.rolling(i).std()
        features['log_pr_std_{}'.format(i)] = log_pr_df.groupby(level='stock').log_pr.apply(std)

    # RSI
    # features['rsi_20'] = log_pr_df.groupby(level='stock').log_pr.apply(rsi)
    features['rsi_30'] = log_pr_df.groupby(level='stock').log_pr.apply(rsi, periods=30)
    # features['rsi_50'] = log_pr_df.groupby(level='stock').log_pr.apply(rsi, periods=50)

    # volume feature
    log_fn = lambda x: np.log(x+1)
    features['log_volu'] = volu_df.groupby(level='stock').volu.apply(log_fn)

    # stdised volume in 2 hours backward rolling windows
    zscore_fn = lambda x: (x - x.rolling(window=240, min_periods=20).mean()) / x.rolling(window=240, min_periods=20).std()
    features['volu_z_score'] = volu_df.groupby(level='stock').volu.apply(zscore_fn)


    # drop min, max features
    features = features.drop(columns=['pr_min_40', 'pr_max_40', 'pr_so_40'])

    response = log_pr.diff(30)

    if grp_idx is not None:
        feature_dict = {}
        for key, idx_lis in grp_idx.items():
            feature_df_dropped = wide_format(features.loc[pd.IndexSlice[:,idx_lis],:])
            # transform back to wide format
            feature_dict[key] = (feature_df_dropped.iloc[x_begin_idx:x_end_idx], 
                                            response[idx_lis].iloc[y_begin_idx:])
        return feature_dict
    else:
        # transform back to wide format
        feature_df_dropped = wide_format(features).iloc[x_begin_idx:x_end_idx]
        # feature_df_dropped = feature_df[x_begin_idx:x_end_idx]
    
        if print_cor:
            for i in range(10):
                
                feature_train_0 = features.xs(i, level='stock').iloc[x_begin_idx:x_end_idx]
                print(feature_train_0.corrwith(response[i]))
                print(feature_train_0.isnull().sum())

        return feature_df_dropped, response.iloc[y_begin_idx:]


In [3]:
grp_idx = {i: [i] for i in range(10)}

x_begin_idx = 41
x_end_idx = -30
y_begin_idx = 71

train_split_t = log_pr.index[-87841]
#vali_split_t = log_pr.index[-44641]

train_feature_dict = get_feature_train(log_pr[:train_split_t], volu[:train_split_t], x_begin_idx, x_end_idx, y_begin_idx,
                                        grp_idx=grp_idx, print_cor=False)

test_feature_dict = get_feature_train(log_pr[train_split_t:], volu[train_split_t:], x_begin_idx, x_end_idx, y_begin_idx,
                                        grp_idx=grp_idx,print_cor=False)

## Model Fitting

### Feature Selection with AIC (Manual)

In [4]:
from sklearn.metrics import mean_squared_error
from statsmodels.regression.linear_model import OLS

In [ ]:
xtrain0, ytrain0 = train_feature_dict[2]
ytrain0 = ytrain0.set_index(xtrain0.index)
#reg0,feature0 = forward_regression(xtrain0,ytrain0)
#reg0.summary()

In [ ]:
xtrain0.columns

### Train function OLS

In [ ]:
def train_OLS(feature_dict):
    mod_dict = {}
    for i, (X, y) in feature_dict.items():
        mod_dict[i] = OLS(y.values, X.values).fit()
    return mod_dict

mod_dict = train_OLS(train_feature_dict)

### Train function Subseted LS

In [5]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import statsmodels.api as sm
import itertools as it

def forward_regression(X, y):
    '''
    Input
    X,y: training matrix (without intercept)
    Return
    model: ols model fitted with intercept on the selected features
    feature_selected: selected features
    '''
    initial_list = []
    included = list(initial_list)
    feature_num = len(X.columns)
    best_bics = pd.Series(index={i for i in range(feature_num)})
    best_features = list(it.repeat([],feature_num))
    for k in range(feature_num):
        excluded = list(set(X.columns)-set(included))
        new_bic = pd.Series(index=excluded)
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_bic[new_column] = model.bic
        best_bic = new_bic.min()
        best_bics[k] = best_bic
        best_feature = new_bic.idxmin()
        included.append(best_feature)
        best_features[k] = included.copy()
    feature_selected = best_features[best_bics.idxmin()]
    model = sm.OLS(y,sm.add_constant(pd.DataFrame(X[feature_selected]))).fit()
    return model,feature_selected

In [7]:
model_dict = {}
feature_dict = {}
for i, (X, y) in train_feature_dict.items():
    y = y.set_index(X.index)
    model, feature = forward_regression(X,y)
    model_dict[i] = model
    feature_dict[i] = feature

In [14]:
feature_dict

{0: ['rsi_30_0'],
 1: ['log_pr_30_1', 'log_pr_std_30_1', 'rsi_30_1', 'log_pr_std_10_1'],
 2: ['volu_z_score_2',
  'rsi_30_2',
  'log_pr_std_10_2',
  'pr_so_40d3_2',
  'log_volu_2'],
 3: ['log_pr_30_3', 'log_pr_std_30_3', 'rsi_30_3', 'log_volu_3'],
 4: ['log_pr_30_4',
  'log_pr_std_30_4',
  'log_volu_4',
  'pr_so_40d3_4',
  'log_pr_std_10_4'],
 5: ['log_pr_std_10_5', 'log_pr_30_5', 'volu_z_score_5'],
 6: ['rsi_30_6',
  'log_volu_6',
  'log_pr_30_6',
  'volu_z_score_6',
  'pr_so_40d3_6',
  'log_pr_std_10_6'],
 7: ['pr_so_40d3_7',
  'log_pr_std_30_7',
  'log_pr_std_10_7',
  'volu_z_score_7',
  'log_volu_7',
  'log_pr_30_7',
  'rsi_30_7'],
 8: ['log_pr_30_8',
  'log_pr_std_10_8',
  'log_pr_std_30_8',
  'rsi_30_8',
  'volu_z_score_8'],
 9: ['log_pr_std_10_9', 'log_pr_std_30_9', 'log_pr_30_9', 'rsi_30_9']}

## Save Model

In [ ]:
import pickle
with open("models.pckl", "wb") as f:
    for model in mod_dict.values():
        pickle.dump(model, f)

In [ ]:
# subsetted models
import pickle
with open("models_subsetls.pckl", "wb") as f:
    for model in mod_dict.values():
        pickle.dump(model, f)

## Evaluation

In [ ]:
# OLS
def wide_format_test(df):
    df_= df.reset_index()
    df_ = df_.pivot(columns ='index').apply(lambda s: s.dropna().reset_index(drop=True))
    df_.columns = df_.columns.get_level_values(0) + '_' +  [str(x) for x in df_.columns.get_level_values(1)]

    return df_

def get_feature_test(log_pr, volu, grp_idx=None):
    """
    Input: 
    log_pr (pdSeries): 1 day of log pr 
    volu (pdSeries): 1 day of volume

    Output:
    test data frame
    """
    features = pd.DataFrame(index=log_pr.columns)

    # backward return
    # print(-(log_pr.iloc[-1] - log_pr.iloc[-30]).values)
    for i in [10, 20, 30]:
        features['log_pr_{}'.format(i)] = -(log_pr.iloc[-1] - log_pr.iloc[-i]).values
    # backward rolling std
    features['log_pr_std_10'] = log_pr.iloc[-10:].std(0).values
    
    # volume features
    features['log_volu'] = np.log(volu.iloc[-1].values + 1)
    features['volu_z_score'] = ((volu.iloc[-1] - volu.iloc[-240:].mean())/volu.iloc[-240:].std()).values

    if grp_idx is None:
        return wide_format_test(features)
    else:
        df_dict = {}
        for key, idx_lis in grp_idx.items():
            df_dict[key] = wide_format_test(features.loc[idx_lis])
        return df_dict

model_dict = mod_dict #{i: pickle.load(open('../model/ridge{}.sav'.format(i), 'rb')) for i in range(2)}

def get_r_hat(A, B): 
    """
        A: 1440-by-10 dataframe of log prices with columns log_pr_0, ... , log_pr_9
        B: 1440-by-10 dataframe of trading volumes with columns volu_0, ... , volu_9    
        return: a numpy array of length 10, corresponding to the predictions for the forward 30-minutes returns of assets 0, 1, 2, ..., 9
    """
    grp_idx = {i: [i] for i in range(10)}
    x = get_feature_test(A, B, grp_idx=grp_idx)
    pred_dict = {i: model.predict(x[i]) for i, model in model_dict.items()}
    
    out = np.zeros(10)
    for keys, idx in grp_idx.items():
        out[idx] = pred_dict.get(keys)
    return out

In [10]:
# Subset LS help function
def rsi_test(log_pr, periods=20):
    """
    Returns a pd.Series with the relative strength index.
    """
    close_delta = log_pr.diff()

    # Make two series: one for lower closes and one for higher closes
    up = close_delta.clip(lower=0)
    down = -1 * close_delta.clip(upper=0)
    
    # Use exponential moving average
    ma_up = up.ewm(com=periods-1, adjust=True, min_periods=periods).mean().iloc[-1]
    ma_down = down.ewm(com=periods-1, adjust=True, min_periods=periods).mean().iloc[-1]
        
    rsi = ma_up / ma_down
    rsi = 100 - (100/(1 + rsi))
    return rsi

def wide_format_test(df):
    df_= df.reset_index()
    df_ = df_.pivot(columns ='index').apply(lambda s: s.dropna().reset_index(drop=True))
    df_.columns = df_.columns.get_level_values(0) + '_' +  [str(x) for x in df_.columns.get_level_values(1)]

    return df_

def get_feature_test(log_pr, volu, grp_idx=None):
    """
    Input: 
    log_pr (pdSeries): 1 day of log pr 
    volu (pdSeries): 1 day of volume

    Output:
    test data frame
    """
    features = pd.DataFrame(index=log_pr.columns)
    features['log_pr_30'] = -(log_pr.iloc[-1] - log_pr.iloc[-31]).values
    
    # Oscilator
    k_period = 40
    d_period = 3
    pr_min_40 = log_pr.rolling(k_period).min().iloc[-d_period:].values
    pr_max_40 = log_pr.rolling(k_period).max().iloc[-d_period:].values
    pr_so_40 = (log_pr.iloc[-d_period:].values - pr_min_40)*100 / (pr_max_40 - pr_min_40)
    features['pr_so_40d3']  = pr_so_40.mean(0)

    # backward rolling std
    features['log_pr_std_10'] = log_pr.iloc[-10:].std(0).values
    features['log_pr_std_30'] = log_pr.iloc[-30:].std(0).values
    
    # RSI
    features['rsi_30'] = log_pr.apply(rsi_test, periods=30)

    # volume features
    features['log_volu'] = np.log(volu.iloc[-1].values + 1)
    features['volu_z_score'] = ((volu.iloc[-1] - volu.iloc[-240:].mean())/volu.iloc[-240:].std()).values

    # print(volu.iloc[-240:].mean())

    if grp_idx is None:
        return wide_format_test(features)
    else:
        df_dict = {}
        for key, idx_lis in grp_idx.items():
            df_dict[key] = wide_format_test(features.loc[idx_lis])[feature_dict[key]]
        return df_dict


def get_r_hat(A, B): 
    """
        A: 1440-by-10 dataframe of log prices with columns log_pr_0, ... , log_pr_9
        B: 1440-by-10 dataframe of trading volumes with columns volu_0, ... , volu_9    
        return: a numpy array of length 10, corresponding to the predictions for the forward 30-minutes returns of assets 0, 1, 2, ..., 9
    """
    grp_idx = {i:[i] for i in range(10)}
    x = get_feature_test(A, B, grp_idx=grp_idx)
    pred_dict = {i: model.predict(np.insert(x[i].values,0,1.)) for i, model in model_dict.items()}
    
    out = np.zeros(10)
    for keys, idx in grp_idx.items():
        out[idx] = pred_dict.get(keys)

    return out


def evaluate_tune(log_pr_test, volu_test):

    t0 = time.time()
    dt = datetime.timedelta(days=1)

    r_fwd = (log_pr_test.shift(-30) - log_pr_test).iloc[1440::10]
    # r_fwd = return_true.iloc[1440::10]
    # r_fwd.index = log_pr_test.index[1440::10]
    r_hat = pd.DataFrame(index=log_pr_test.index[1440::10], columns=log_pr_test.columns, dtype=np.float64)

    for t in log_pr_test.index[1440::10]: # compute the predictions every 10 minutes
        # inputs 1 day of log price and volume
        r_hat.loc[t, :] = get_r_hat(log_pr_test.loc[(t - dt):t], volu_test.loc[(t - dt):t])
    t_used = time.time() - t0
    print("Time used: ", t_used)

    r_fwd_all = r_fwd.iloc[:-3].values.ravel() # the final 3 rows are NaNs. 
    r_hat_all = r_hat.iloc[:-3].values.ravel()
    return r_fwd.corrwith(r_hat), np.corrcoef(r_fwd_all, r_hat_all)[0,1]

In [13]:
log_pr_test = log_pr[train_split_t:]
volu_test = volu[train_split_t:]
evaluate_tune(log_pr_test, volu_test)

Time used:  790.7790622711182


(0   -0.013158
 1    0.021653
 2    0.027461
 3    0.001490
 4    0.117057
 5    0.026351
 6    0.020502
 7    0.010828
 8    0.106681
 9    0.049929
 dtype: float64,
 0.03897242904209)

In [ ]:
log_pr_test = log_pr
volu_test = volu

In [ ]:
evaluate_tune(log_pr_test, volu_test)